In [ ]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as k
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import obtain_input_shape

In [ ]:
import tensorflow as tf
tf.__version__

!sudo pip3 install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.layers import Layer, InputSpec

In [ ]:
def VGGupdated(input_tensor=None, classes=2): 
    
    img_rows, img_cols = 224,224
    img_channels = 3 #we are going to work on coloured images
    
    img_dim = (img_rows, img_cols, img_channels) 
    
    img_input = Input(shape = img_dim) #input from user we want is tis
    
    
    #block1
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='block1_conv1')(img_input) #64 filters of 3x3,padding same as we want size of output image ame as input image
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='block1_conv2')(x)#relu will replace all the negative values with 0
    x = MaxPooling2D((2,2), strides=(2,2), name='block1_pool')(x) #pooling to reduce the image size, strides the window size for scanning
    
    #block2
    x = Conv2D(128, (3,3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2,2), strides=(2,2), name='block2_pool')(x) 
    
    #block3
    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2,2), strides=(2,2), name='block3_pool')(x)
    
    #block4
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2,2), strides=(2,2), name='block4_pool')(x)
    
    #block5
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2,2), strides=(2,2), name='block5_pool')(x)
    
    #classification block
    x = Flatten(name='flatten')(x) #when we need to classify image we ned to flaten it
    x = Dense(4096, activation='relu', name='fc1')(x) #4096 nurons
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='sigmoid', name='predictions')(x)
    
    
    #create model
    model = Model(inputs = img_input, outputs = x, name='vggtuber')
    
    return model

In [ ]:
model = VGGupdated(classes = 2) # normal and tuberculosis

In [ ]:
model.summary()

Model: "vggtuber"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

In [ ]:
model.compile(
  loss='SparseCategoricalCrossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/tuber_images') #will give the list of folders in the tuber_images

x_ray_types = os.listdir('/content/drive/MyDrive/tuber_images')
print(x_ray_types)# what types of xray are there in dataset

print("types of xray found:", len(dataset_path))

['Tuberculosis', 'Normal']
types of xray found: 2


In [ ]:
rays = []

for item in x_ray_types:
    all_rays = os.listdir('/content/drive/MyDrive/tuber_images' + '/' + item)
    
    
    for ray in all_rays:
        rays.append((item, str('/content/drive/MyDrive/tuber_images' + '/' + item) + '/' + ray))
        print(rays)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
rooms_df = pd.DataFrame(data=rays, columns=['x_ray types', 'image'])
print(rooms_df.head())

    x_ray types                                              image
0  Tuberculosis  /content/drive/MyDrive/tuber_images/Tuberculos...
1  Tuberculosis  /content/drive/MyDrive/tuber_images/Tuberculos...
2  Tuberculosis  /content/drive/MyDrive/tuber_images/Tuberculos...
3  Tuberculosis  /content/drive/MyDrive/tuber_images/Tuberculos...
4  Tuberculosis  /content/drive/MyDrive/tuber_images/Tuberculos...


In [ ]:
print("Total number of images in dataset: ", len(rooms_df))

room_count = rooms_df['x_ray types'].value_counts()

print("xrays in each category: ")
print(room_count)

Total number of images in dataset:  4280
xrays in each category: 
Normal          3575
Tuberculosis     705
Name: x_ray types, dtype: int64


In [ ]:
import cv2
path = '/content/drive/MyDrive/tuber_images/'

im_size=224

images = []
labels=[]

for i in x_ray_types:
    data_path = path +str(i)
    filenames = [i for i in os.listdir(data_path)]
    
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [ ]:
images = np.array(images)

images = images.astype('float32')/255.0
images.shape

(4280, 224, 224, 3)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y=rooms_df['x_ray types'].values

y_labelencoder = LabelEncoder()
y= y_labelencoder.fit_transform (y)
print(y)

[1 1 1 ... 0 0 0]


In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

y = shuffle(y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split( images, y, test_size=0.15, random_state=415)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(3638, 224, 224, 3)
(3638,)
(642, 224, 224, 3)
(642,)


In [ ]:
model.fit(train_x, train_y, epochs = 5, batch_size=32)

Epoch 1/5
114/114 [==============================] - 769s 7s/step - loss: 0.5045 - accuracy: 0.8321
Epoch 2/5
114/114 [==============================] - 1139s 10s/step - loss: 0.4496 - accuracy: 0.8367
Epoch 3/5
114/114 [==============================] - 1113s 10s/step - loss: 0.4511 - accuracy: 0.8367
Epoch 4/5
114/114 [==============================] - 1133s 10s/step - loss: 0.4484 - accuracy: 0.8367
Epoch 5/5
114/114 [==============================] - 1115s 10s/step - loss: 0.4475 - accuracy: 0.8367


In [ ]:
preds = model.evaluate(test_x, test_y)
print("loss = " + str(preds[0]))
print("Test accuracy= " + str(preds[1]))

21/21 [==============================] - 24s 1s/step - loss: 0.4605 - accuracy: 0.8271
loss = 0.46049752831459045
Test accuracy= 0.827102780342102
